In [127]:
import numpy as np
import csv

In [144]:
class Team:
    
    def __init__(self,name,seed,region,probs):
        
        self.name = name
        self.seed = seed**seed_pow
        self.probs = probs
        self.region = region
    
    
class Game:
    
    def __init__(self):
        
        self.teams = [0,0]
        self.pred_winner = 0
        self.winner_round_out = 6
        self.prev_games = [0,0]
        self.next_game = 0
        self.game_num = 0
        self.round_num = 0
    
    def score(self,team_num,gamma,weights):
        
        team = self.teams[team_num]
        prob_winner_to_out_round = sum(team.probs[self.winner_round_out:])
        probs = team.probs[:self.winner_round_out] + [prob_winner_to_out_round]
        
        w_sums = np.cumsum(weights[:self.winner_round_out+1])
        
        EX = team.seed*np.dot(w_sums,probs)
        E_X2 = team.seed**2*np.dot(w_sums*w_sums,probs)
        
        std_dev = np.sqrt(E_X2 - EX**2)
        
        return EX + gamma*std_dev
    
    def set_winner_out(self,losing_team,round_num):
        
        if self.round_num == 0:
            return
        
        for game in self.prev_games:
            if losing_team in game.teams:
                game.winner_round_out = round_num
                game.set_winner_out(losing_team,round_num)
            
        return
    
    def predict_game(self,gamma,weights):
        
        if teams[0] == 0 or teams[1] == 0:
            print('Teams not Defined for Game',
                  self.game_num,
                  'Round',
                  self.round_num)
            return
        
        score_1 = self.score(0,gamma,weights)
        score_2 = self.score(1,gamma,weights)
        
        if score_1 > score_2:
            self.pred_winner = self.teams[0]
            self.set_winner_out(self.teams[1],self.round_num)
        else:
            self.pred_winner = self.teams[1]
            self.set_winner_out(self.teams[0],self.round_num)
        
        if self.round_num != 5:
            if self.game_num % 2 == 0:
                self.next_game.teams[0] = self.pred_winner
            else:
                self.next_game.teams[1] = self.pred_winner
        
        return
    
    
class Round:
    
    def __init__(self,round_num):
        
        self.round_num = round_num
        self.games = [Game() for i in range(int(32/(2**round_num)))]
        
    def predict_round(self,gamma,weights):
        
        for game in self.games:
            
            if game == 0:
                print('Games Not Defined in Round', self.round_num)
                return
            
            game.predict_game(gamma,weights)
    
        return
        

class Bracket:
    
    def __init__(self,teams,weights):
        
        self.rounds = [Round(i) for i in range(6)]
            
        for rnd in self.rounds:
            i = 0
            
            for game in rnd.games:
                
                game.game_num = i
                game.round_num = rnd.round_num
                
                if rnd.round_num == 0:
                    game.teams = teams[2*i:(2*i+2)]
                
                if rnd.round_num  != 5:
                    game.next_game = self.rounds[rnd.round_num+1].games[int(np.floor(i/2))]
                    
                if rnd.round_num != 0:
                    game.prev_games = self.rounds[rnd.round_num-1].games[2*i:(2*i+2)]
                
                i = i+1
                
    def predict_bracket(self,gamma,weights):
        
        for rnd in self.rounds:
            rnd.predict_round(gamma,weights)
            
        return
    
    def print_teams(self):
        for rnd in self.rounds:
            print("")
            print("Round", rnd.round_num,"winners:")
            print("")
            for game in rnd.games:
                print(game.pred_winner.name)

In [168]:
def process_probs(probs):
    
    new_probs = [(1-probs[0])]
    for i in range(len(probs)-1):
        prob_out = probs[i]*(1-(probs[i+1]/probs[i]))
        new_probs.append(prob_out)
    new_probs.append(probs[-1])
        
    return new_probs

In [170]:
seed_pow = 0.25
weights = [0,1,2,4,8,16,32]
var_pen = -0.25
pred_iterations = 3

teams = []
with open('probs.csv') as f:
    reader = csv.reader(f)
    for row in reader:
        if row[0] == 'mens' and \
            row[1] == '2018-03-14' and \
            int(row[10]) == 1:
                
            probs = list(map(float, row[4:10]))
            probs = process_probs(probs)
            name = row[12]
            if row[-1] == '11a' or row[-1] == '11b':
                seed = 11
            elif row[-1] == '16a' or row[-1] == '16b':
                seed = 16
            else:
                seed = int(row[-1])
                
            region = row[-2]
            teams.append(Team(name,seed,region,probs))
            
seed_order = np.power([1,16,8,9,5,12,4,13,6,11,3,14,7,10,2,15],seed_pow)
region_order = ['South','West','East','Midwest']
teams_sorted = []
for region in region_order:
    for seed in seed_order:
        for team in teams:
            if team.region == region and team.seed == seed:
                teams_sorted.append(team)
                
Brac = Bracket(teams_sorted,weights)
for _ in range(pred_iterations):
    Brac.predict_bracket(var_pen,weights)
    
Brac.print_teams()


Round 0 winners:

Virginia
Creighton
Kentucky
Arizona
Miami (FL)
Tennessee
Texas
Cincinnati
Xavier
Florida State
Ohio State
Gonzaga
Houston
Michigan
Texas A&M
North Carolina
Villanova
Virginia Tech
West Virginia
Wichita State
Florida
Texas Tech
Butler
Purdue
Kansas
Seton Hall
Clemson
Auburn
Texas Christian
Michigan State
Rhode Island
Duke

Round 1 winners:

Virginia
Arizona
Tennessee
Cincinnati
Xavier
Gonzaga
Michigan
North Carolina
Villanova
Wichita State
Texas Tech
Purdue
Kansas
Auburn
Michigan State
Duke

Round 2 winners:

Virginia
Cincinnati
Gonzaga
North Carolina
Villanova
Purdue
Kansas
Duke

Round 3 winners:

Virginia
Gonzaga
Villanova
Duke

Round 4 winners:

Virginia
Villanova

Round 5 winners:

Villanova
